In [1]:
import pickle
import mlflow
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)
    
from sklearn.pipeline import make_pipeline

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("green-taxi-duration")

2023/06/25 17:02:52 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/06/25 17:02:52 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/home/hilbert/zoomcamps/4_deployment/prefect-batch/mlruns/1', creation_time=1687705373156, experiment_id='1', last_update_time=1687705373156, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [2]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df["duration"] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

def prepare_dictionaries(df):
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    dicts = df[categorical + numerical].to_dict(orient="records")

    return dicts
    
dftr = read_dataframe("../../data/green_tripdata_2022-01.parquet")
dfvl = read_dataframe("../../data/green_tripdata_2022-02.parquet")

target = "duration"
ytr = dftr[target].values
yvl = dfvl[target].values

dict_train = prepare_dictionaries(dftr)
dict_val = prepare_dictionaries(dfvl)

In [3]:
with mlflow.start_run():

    params = dict(
        max_depth=20,
        n_estimators=100,
        min_samples_leaf=10,
        random_state=0
    )

    mlflow.log_params(params)
   
    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, ytr)
    ypr = pipeline.predict(dict_val)

    rmse = mean_squared_error(ypr, yvl, squared=False)
    print(params, rmse)
    mlflow.log_metric("rmse", rmse)

    mlflow.sklearn.log_model(pipeline, "model")



{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.101201727016495


/home/hilbert/anaconda3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
